In [1]:
import mido
TICKS_PER_BEAT = 480
TEMPO = int(mido.bpm2tempo(120))

In [40]:
%run 'get_training_data.ipynb'

 > 25/100 files loaded.
Training data too short: PkmRB-Item2.mid. len(vector)=50
 > 50/100 files loaded.
 > 75/100 files loaded.
 > 100/100 files loaded.


In [35]:
def get_track_sequences(vector_sequence):
    """Get note sequences for each track from vector sequence output from RNN.

    Args:
       vector_sequence: List of one-hot vectors containing 128 * NUM_TRACKS note_on events, 128 * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 1/4 of a beat each.
       
    Returns:
        List of track vector sequences.
    """
    track_sequences = [[] for _ in range(NUM_TRACKS)]
    start_time = 0
    for vector in vector_sequence:
        index = np.nonzero(vector == 1)[0][0]
        # vector is a time event.
        if index >= NUM_MIDI_PITCHES * 2 * NUM_TRACKS:
            num_beats = (index - NUM_MIDI_PITCHES * 2 * NUM_TRACKS + 1) / NUM_TIMESHIFTS
            start_time += num_beats
        else:
            # vector is a note_off event.
            if index >= NUM_MIDI_PITCHES * NUM_TRACKS:
                note_type = 'note_off'
                index -= NUM_MIDI_PITCHES * NUM_TRACKS
            # vector is a note_on event.
            else:
                note_type = 'note_on'
            track_num = int(index / NUM_MIDI_PITCHES)
            note = index % NUM_MIDI_PITCHES
            track_sequences[track_num].append({"type": note_type, "note": note, "start_time": start_time})
            time_delay = 0
            
    return track_sequences
    

In [36]:
def get_midi_from_vector_sequence(track_sequences):
    """Get midi from track sequences.

    Args:
       track_sequences: List of vector sequences for each track.
       
    Returns:
        MidiFile object.
    """
    mid = mido.MidiFile()
    tracks = [mido.MidiTrack() for _ in range(NUM_TRACKS)]
    mid.tracks.extend(tracks)

    for i, ts in enumerate(track_sequences):
        prev_start_time = 0
        for event in ts:
            time = event["start_time"] - prev_start_time
            prev_start_time = event["start_time"]
            # I picked a random number for velocity.
            mid.tracks[i].append(mido.Message(event["type"], note=event["note"], velocity=50,
                                              time=int(time * TICKS_PER_BEAT)))
    
    return mid

In [37]:
def vector_to_midi(vector_sequence):
    """Get midi file from vector sequence output from RNN.

    Args:
       vector_sequence: List of one-hot vectors containing 128 * NUM_TRACKS note_on events, 128 * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 1/4 of a beat each.
       
    Returns:
        MidiFile object.
    """
    track_sequences = get_track_sequences(vector_sequence)
    return get_midi_from_vector_sequence(track_sequences)

In [10]:
training_filename = 'training_data.txt'
get_training_data(0, 6000, training_filename=training_filename)
get_training_vectors_from_file(training_filename)

 > 25/5824 files loaded.
Training data too short: PkmRB-Item2.mid. len(vector)=50
 > 50/5824 files loaded.
 > 75/5824 files loaded.
 > 100/5824 files loaded.
Training data too short: trojintr.mid. len(vector)=87
 > 125/5824 files loaded.
Faulty training data: DG_SMB_Main.mid. len(vector)=1179
 > 150/5824 files loaded.
 > 175/5824 files loaded.
 > 200/5824 files loaded.
 > 225/5824 files loaded.
 > 250/5824 files loaded.
Training data too short: Gpunchoutboss.mid. len(vector)=72
 > 275/5824 files loaded.
Training data too short: Zelda_-_Game_Over.mid. len(vector)=96
 > 300/5824 files loaded.
Training data too short: immortatatk.mid. len(vector)=45
 > 325/5824 files loaded.
 > 350/5824 files loaded.
 > 375/5824 files loaded.
Training data too short: Flag.mid. len(vector)=81
 > 400/5824 files loaded.
Training data too short: metroidit.mid. len(vector)=59
 > 425/5824 files loaded.
 > 450/5824 files loaded.
Training data too short: rcproam_startrace.mid. len(vector)=97
 > 475/5824 files loa

 > 4575/5824 files loaded.
Training data too short: Mm6vict2.mid. len(vector)=94
 > 4600/5824 files loaded.
 > 4625/5824 files loaded.
 > 4650/5824 files loaded.
 > 4675/5824 files loaded.
 > 4700/5824 files loaded.
 > 4725/5824 files loaded.
Training data too short: cvLostLife.mid. len(vector)=64
 > 4750/5824 files loaded.
Training data too short: Metrma.mid. len(vector)=77
 > 4775/5824 files loaded.
 > 4800/5824 files loaded.
 > 4825/5824 files loaded.
 > 4850/5824 files loaded.
Training data too short: zelda-go.mid. len(vector)=70
Faulty training data: DG_MM5_Darkman.mid. len(vector)=374
 > 4875/5824 files loaded.
Training data too short: mm3protw.mid. len(vector)=80
 > 4900/5824 files loaded.
 > 4925/5824 files loaded.
Training data too short: POtbout.mid. len(vector)=72
 > 4950/5824 files loaded.
Training data too short: Deleting_Photo.mid. len(vector)=75
 > 4975/5824 files loaded.
 > 5000/5824 files loaded.
Training data too short: STANLevel_Complete.mid. len(vector)=75
Training 

(array([[7],
        [7],
        [4],
        ..., 
        [7],
        [7],
        [1]]),
 ['FridayThe13th_-_MapDark.mid',
  'gymleader2.mid',
  'Zelda_IV_-_Tail_Cave.mid',
  'cv1-4.mid',
  'MM3-Snake_Man.mid',
  'smlboss.mid',
  'molemania_snow.mid',
  'dw4batl.mid',
  'mm1wily2.mid',
  'dd-intermission.mid',
  '1999.mid',
  'FiremanByCryogen.mid',
  'Heavy_Barrel_-_Boss_BGM.mid',
  'Train.mid',
  'Wizards_and_Warriors_II_Ironsword_-_Wind_Elemental.mid',
  'DK-07-Iceberg.mid',
  'supercend.mid',
  'DW4TALN.mid',
  'shatter2_v10.mid',
  'Spark_Man.mid',
  'boss-drums.mid',
  'T_NinjaGaidenShadow_Stage3.mid',
  'dd2-mission5.mid',
  'Grad21_2_v2.mid',
  'elm_street_metal.mid',
  'Sparkman6969.mid',
  'Rick_Rude.mid',
  '12-Magnetman.mid',
  'Bbtitle2.mid',
  'WizWar2Lvl1.mid',
  'StarTropics-alien_ship.mid',
  'Gbboss.mid',
  'bohmagna.mid',
  'wl2end.mid',
  'TheLegendRemixed.mid',
  'lf-st4.mid',
  'IcarusGBW1.mid',
  'gradius7.mid',
  'Klxtitle.mid',
  'CentaurmanFulminantMix.mid

In [43]:
def text_vector_to_midi(vector):
    """Generate midi from text vector.
    
    Args:
       vector: Vector with the index of the one-hot vector.
       
    Returns:
        MidiFile object.
    """
    ONE_HOT_MATRIX = np.eye(NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
    vector_sequence = []
    for v in vector:
        vector_sequence.append(ONE_HOT_MATRIX[v])
    return vector_to_midi(vector_sequence)

In [44]:
vectors, names = get_training_vectors_from_file(training_filename)
name = '1999.mid'
index = names.index(name)
mid = text_vector_to_midi(vectors[index])
mid.save('txt' + name)